In [2]:
# Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import time
from datetime import datetime, timedelta, date
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_columns', None)

In [15]:
cdc_cases_deaths_csv = "https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD&bom=true&format=true"
us_tests_csv = "https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/testing_data/time_series_covid19_US.csv?raw=true"

# Folders
repositoryFolder = "D:/Repositories/Global-COVID-Surveillance/data/"
localDownloadFolder = "C:/Users/janin/Downloads/"
demographicsFolder = repositoryFolder + "raw/demographics/"
configuredFolder = repositoryFolder + "configured/"
cleanedFolder = repositoryFolder + "cleaned/"
regionsFolder = repositoryFolder + "raw/regions/"
locationsFolder = repositoryFolder + "raw/locations/"
rFolder = localDownloadFolder
dataFolder = repositoryFolder

# Population Input File
all_populations_file = cleanedFolder + "all_populations.xlsx"
us_codes = demographicsFolder + "US State Codes.xlsx"

In [29]:
census_regions = {
    0: {"name" : "United States",
        "states" : ["United States"]},
    1: {"name" : "Northeast",
        "states" :["Connecticut", "Maine", "New Hampshire", "Vermont", "Massachusetts", 
                   "Rhode Island", "New Jersey", "New York", "Pennsylvania"]},
    3: {"name" : "South",
        "states" : ["Maryland", "Delaware", "West Virginia", "Virginia", "Kentucky", 
                    "Tennessee", "North Carolina", "South Carolina", "Georgia", "Florida", 
                    "Alabama", "Mississippi", "Arkansas", "Louisiana", "Oklahoma", "Texas", 
                    "District of Columbia", "Puerto Rico"]},
    2: {"name" : "Midwest",
        "states" : ["North Dakota", "South Dakota", "Nebraska", "Kansas", "Missouri", "Iowa", 
                    "Minnesota", "Wisconsin", "Illinois", "Michigan", "Indiana", "Ohio"]},
    4: {"name" : "West",
        "states" : ["Washington", "Idaho", "Montana", "Wyoming", "Oregon", "California", "Nevada", 
                    "Utah", "Colorado", "Arizona", "New Mexico", "Alaska", "Hawaii"]}
}

us_states = [
    'Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia',
    'Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
    'Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska',
    'Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota',
    'Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas',
    'Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming'
]
canada_provinces = [
    'Alberta','British Columbia','Manitoba','New Brunswick','Newfoundland and Labrador','Northwest Territories',
    'Nova Scotia','Ontario','Prince Edward Island','Quebec','Saskatchewan','Yukon'
]
states_and_provinces = us_states + canada_provinces
us_territories =['American Samoa','Commonwealth of the Northern Mariana Islands','Guam','Puerto Rico','U.S. Virgin Islands']

In [11]:
def censusRegionByState(state):
    for key in census_regions.keys():
        if (state in census_regions[key]["states"]):
            return census_regions[key]["name"]
        
def fixUSRegion(code):
    region = census_regions[code]
    region_name = region["name"]
    return region_name

def convertDateToExcel(dayString) :
    temp = datetime(1899, 12, 30)
    day = datetime.strptime(dayString, '%m/%d/%Y')
    delta = day - temp
    return float(delta.days) + (float(delta.seconds) / 86400)

In [14]:
all_populations = pd.read_excel(cleanedFolder + "all_populations.xlsx")
population_groups = ["Level","Region","Census Region","Country","State/Province"]
for group in population_groups:
    all_populations[group] = all_populations[group].astype(str)
    all_populations[group] = all_populations[group].apply(lambda x: "" if x=="nan" else x)
print(all_populations.columns)
all_populations.head()

Index([                  'Level',                  'Region',
                       'Country',           'Census Region',
                'State/Province',              'Population',
               'Population 100K',      'Country Population',
       'Country Population 100K',       'Region Population',
       ...
                       'Pct 1-4',                'Pct 5-14',
                     'Pct 15-24',               'Pct 25-34',
                     'Pct 35-44',               'Pct 45-54',
                     'Pct 55-64',               'Pct 65-74',
                     'Pct 75-84',                 'Pct 85+'],
      dtype='object', length=131)


,Level,Region,Country,Census Region,State/Province,Population,Population 100K,Country Population,Country Population 100K,Region Population,Region Population 100K,World Population,World Population 100K,Country Share,Region Share,World Share,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
0,Country,Central Asia,Armenia,,,2963243,29.63243,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.009065,0.000380,0.04,63.0,0.19,5512.0,-4998.0,104.0,28470.0,1.8,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country,Central Asia,Azerbaijan,,,10139177,101.39177,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.031017,0.001300,0.13,56.0,0.91,91459.0,1200.0,123.0,82658.0,2.1,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Country,Central Asia,Cyprus,,,1207359,12.07359,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.003693,0.000155,0.02,67.0,0.73,8784.0,5000.0,131.0,9240.0,1.3,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country,Central Asia,Faeroe Islands,,,48863,0.48863,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.000149,0.000006,0.00,43.0,0.38,185.0,NaN,35.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Country,Central Asia,Georgia,,,3989167,39.89167,NaN,NaN,326887719,3268.87719,7796609105,77966.09105,NaN,0.012203,0.000512,0.05,58.0,-0.19,-7598.0,-10000.0,57.0,69490.0,2.1,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
us_state_codes = pd.read_excel(us_codes)
us_state_codes["Census Region"] = us_state_codes["State Name"].apply(lambda x: censusRegionByState(x))
us_state_codes.rename(columns = {
    'State Name':'State/Province',
    "State Abbreviation": "Abbreviation"
}, inplace = True)
us_state_codes.head()

,State/Province,FIPS,Abbreviation,Status,Census Region
0,United States,0,US,0,United States
1,Alabama,1,AL,0,South
2,Alaska,2,AK,0,West
3,Arizona,4,AZ,0,West
4,Arkansas,5,AR,0,South


In [23]:
cdc_cases_deaths_request=requests.get(cdc_cases_deaths_csv).content
cdc_cases_deaths=pd.read_csv(io.StringIO(cdc_cases_deaths_request.decode('utf-8')))
currentTime = datetime.now()

print("Original Columns:")
print(cdc_cases_deaths.columns)

cdc_cases_deaths.drop(columns=[
    'created_at',
    'conf_cases', 'prob_cases', 'pnew_case',
    'conf_death', 'prob_death', 'pnew_death', 
    'consent_cases', 'consent_deaths'
], inplace = True)

cdc_cases_deaths["Time"] = cdc_cases_deaths["submission_date"].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
cdc_cases_deaths = cdc_cases_deaths.sort_values(by=['state','Time']).reset_index().drop(columns=['index'])
cdc_cases_deaths.rename(
    columns = {
        'submission_date': 'Date', 'state' : 'Abbreviation',
        'tot_cases' : 'Total Cases', 'new_case' : 'Cases Daily',
        'tot_death' : 'Total Deaths', 'new_death' : 'Deaths Daily'
    }, inplace = True)
cdc_cases_deaths["Accessed"] = currentTime
cdc_cases_deaths["Country"] = "United States"
cdc_cases_deaths["Region"] = "North America"

print("Renamed Columns:")
print(cdc_cases_deaths.columns)
cdc_cases_deaths = cdc_cases_deaths[cdc_cases_deaths_order]
cdc_cases_deaths.head(75)

Original Columns:
Index(['submission_date', 'state', 'tot_cases', 'conf_cases', 'prob_cases',
       'new_case', 'pnew_case', 'tot_death', 'conf_death', 'prob_death',
       'new_death', 'pnew_death', 'created_at', 'consent_cases',
       'consent_deaths'],
      dtype='object')
Renamed Columns:
Index(['Date', 'Abbreviation', 'Total Cases', 'Cases Daily', 'Total Deaths',
       'Deaths Daily', 'Time', 'Accessed', 'Country', 'Region'],
      dtype='object')


,Region,Country,Abbreviation,Time,Date,Cases Daily,Total Cases,Deaths Daily,Total Deaths,Accessed
0,North America,United States,AK,2020-01-22,01/22/2020,0,0,0,0,2021-03-14 14:46:21.129247
1,North America,United States,AK,2020-01-23,01/23/2020,0,0,0,0,2021-03-14 14:46:21.129247
2,North America,United States,AK,2020-01-24,01/24/2020,0,0,0,0,2021-03-14 14:46:21.129247
3,North America,United States,AK,2020-01-25,01/25/2020,0,0,0,0,2021-03-14 14:46:21.129247
4,North America,United States,AK,2020-01-26,01/26/2020,0,0,0,0,2021-03-14 14:46:21.129247
...,...,...,...,...,...,...,...,...,...,...
70,North America,United States,AK,2020-04-01,04/01/2020,10,143,0,3,2021-03-14 14:46:21.129247
71,North America,United States,AK,2020-04-02,04/02/2020,4,147,0,3,2021-03-14 14:46:21.129247
72,North America,United States,AK,2020-04-03,04/03/2020,10,157,0,3,2021-03-14 14:46:21.129247
73,North America,United States,AK,2020-04-04,04/04/2020,14,171,2,5,2021-03-14 14:46:21.129247


In [60]:
us_tests_request = requests.get(us_tests_csv).content
us_tests = pd.read_csv(io.StringIO(us_tests_request.decode('utf-8')))
print("Original Tests:")
print(us_tests.columns)
us_tests["Time"] = us_tests["date"].apply(lambda x: datetime.strptime(x, '%m/%d/%y'))
us_tests.drop(columns=[
    'date',
    'cases_conf_probable', 
    'tests_viral_positive', 'tests_viral_negative', 
    'people_viral_positive', 'people_viral_total', 
    'encounters_viral_total', 'tests_combined_total'
], inplace = True)
us_tests.rename(
    columns = {
        'date': 'Date', 
        'state' : 'Abbreviation',
        'tests_viral_total' : 'Total Tests'
    }, inplace = True)
us_tests = us_tests.sort_values(by=['Abbreviation','Time']).reset_index().drop(columns=['index'])
us_tests["Tests Daily"] = us_tests.groupby(['Abbreviation'], as_index=False)['Total Tests'].diff().reset_index(drop=True).fillna(0)
print('Renamed Tests')
print(us_tests.columns)
us_tests_order = ['Abbreviation', 'Time', 'Tests Daily', 'Total Tests']
us_tests = us_tests[us_tests_order]
us_tests_min = us_tests["Time"].min()
print("Min Date: " + str(us_tests_min))
us_tests_max = us_tests["Time"].max()
print("Max Date: " + str(us_tests_max))
us_tests.head()

Original Tests:
Index(['date', 'state', 'cases_conf_probable', 'tests_viral_positive',
       'tests_viral_negative', 'tests_viral_total', 'people_viral_positive',
       'people_viral_total', 'encounters_viral_total', 'tests_combined_total'],
      dtype='object')
Renamed Tests
Index(['Abbreviation', 'Total Tests', 'Time', 'Tests Daily'], dtype='object')
Min Date: 2020-01-13 00:00:00
Max Date: 2021-03-13 00:00:00


,Abbreviation,Time,Tests Daily,Total Tests
0,AK,2020-03-06,0.0,8.0
1,AK,2020-03-07,4.0,12.0
2,AK,2020-03-08,2.0,14.0
3,AK,2020-03-09,9.0,23.0
4,AK,2020-03-10,0.0,23.0


In [70]:
states_input = pd.merge(cdc_cases_deaths, us_state_codes, how="left", on="Abbreviation")
states_input = pd.merge(states_input, us_tests, how="left", on=["Abbreviation","Time"])
print(states_input.columns)
states_input["Level"] = states_input.apply(
    lambda x:
    "Territory" if x["State/Province"] in us_territories else 
    "State/Province",
    axis=1
)
states_integers = [
    'Cases Daily', 'Total Cases', 
    'Deaths Daily', 'Total Deaths',
    'Tests Daily', 'Total Tests',
]
float_cols = states_input.select_dtypes(include=['float64'])
for col in float_cols.columns.values:
    states_input[col] = states_input[col].fillna(0).astype('int64')
us_order = [
    'Level', 'Region', 'Country', 'Census Region', 'State/Province', 'Abbreviation', 'FIPS', 
    'Time', 'Date', 'Status', 'Accessed',
    'Cases Daily', 'Total Cases', 
    'Deaths Daily', 'Total Deaths',
    'Tests Daily', 'Total Tests'
]
states_input = states_input[us_order]
states_input_min = states_input["Time"].min()
print("Min Date: " + str(states_input_min))
states_input_max = states_input["Time"].max()
print("Max Date: " + str(states_input_max))
states_input = states_input.merge(all_populations,how="left",on=["Level","Region","Country","Census Region","State/Province"]).reset_index().drop(columns=["index"])
states_input.head(99)

Index(['Region', 'Country', 'Abbreviation', 'Time', 'Date', 'Cases Daily',
       'Total Cases', 'Deaths Daily', 'Total Deaths', 'Accessed',
       'State/Province', 'FIPS', 'Status', 'Census Region', 'Tests Daily',
       'Total Tests'],
      dtype='object')
Min Date: 2020-01-22 00:00:00
Max Date: 2021-03-13 00:00:00


,Level,Region,Country,Census Region,State/Province,Abbreviation,FIPS,Time,Date,Status,Accessed,Cases Daily,Total Cases,Deaths Daily,Total Deaths,Tests Daily,Total Tests,Population,Population 100K,Country Population,Country Population 100K,Region Population,Region Population 100K,World Population,World Population 100K,Country Share,Region Share,World Share,World Share (%),Urban Population (%),Annual Change (%),Net Change,Migrants (net),Density (P/Km²),Land Area (Km²),Fertility Rate,Median Age,< 1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85+,1-4,5-14,15-24,25-34,35-44,45-54,55-64,65-74,75-84,Pct < 1,Pct 1-4,Pct 5-14,Pct 15-24,Pct 25-34,Pct 35-44,Pct 45-54,Pct 55-64,Pct 65-74,Pct 75-84,Pct 85+
0,State/Province,North America,United States,West,Alaska,AK,2,2020-01-22,01/22/2020,0,2021-03-14 14:46:21.129247,0,0,0,0,0,0,712114.0,7.12114,327052602.0,3270.52602,365057840.0,3650.5784,7.796609e+09,77966.09105,0.002177,0.001951,0.000091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9978.0,10012.0,10186.0,10509.0,10395.0,10414.0,10303.0,10286.0,10436.0,10157.0,9976.0,10016.0,9887.0,9509.0,9678.0,9488.0,9410.0,9343.0,8518.0,7525.0,8088.0,8617.0,9132.0,9252.0,9900.0,10318.0,10693.0,11456.0,11576.0,11552.0,10946.0,10809.0,10460.0,10822.0,10799.0,10303.0,10452.0,9962.0,9667.0,9685.0,8865.0,8589.0,8533.0,7954.0,8295.0,7827.0,7962.0,8230.0,8696.0,9086.0,8428.0,8197.0,8330.0,8595.0,9128.0,9426.0,9493.0,9636.0,9731.0,9894.0,9373.0,9168.0,9161.0,8614.0,8800.0,8210.0,7734.0,7220.0,6655.0,6442.0,5978.0,5621.0,5482.0,4013.0,3945.0,3665.0,3459.0,2950.0,2677.0,2327.0,1971.0,1784.0,1586.0,1411.0,1277.0,7181.0,41102.0,100662.0,89273.0,109431.0,92305.0,84479.0,93296.0,61300.0,23107.0,0.014012,0.057718,0.141357,0.125363,0.153671,0.129621,0.118631,0.131013,0.086082,0.032448,0.010084
1,State/Province,North America,United States,West,Alaska,AK,2,2020-01-23,01/23/2020,0,2021-03-14 14:46:21.129247,0,0,0,0,0,0,712114.0,7.12114,327052602.0,3270.52602,365057840.0,3650.5784,7.796609e+09,77966.09105,0.002177,0.001951,0.000091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9978.0,10012.0,10186.0,10509.0,10395.0,10414.0,10303.0,10286.0,10436.0,10157.0,9976.0,10016.0,9887.0,9509.0,9678.0,9488.0,9410.0,9343.0,8518.0,7525.0,8088.0,8617.0,9132.0,9252.0,9900.0,10318.0,10693.0,11456.0,11576.0,11552.0,10946.0,10809.0,10460.0,10822.0,10799.0,10303.0,10452.0,9962.0,9667.0,9685.0,8865.0,8589.0,8533.0,7954.0,8295.0,7827.0,7962.0,8230.0,8696.0,9086.0,8428.0,8197.0,8330.0,8595.0,9128.0,9426.0,9493.0,9636.0,9731.0,9894.0,9373.0,9168.0,9161.0,8614.0,8800.0,8210.0,7734.0,7220.0,6655.0,6442.0,5978.0,5621.0,5482.0,4013.0,3945.0,3665.0,3459.0,2950.0,2677.0,2327.0,1971.0,1784.0,1586.0,1411.0,1277.0,7181.0,41102.0,100662.0,89273.0,109431.0,92305.0,84479.0,93296.0,61300.0,23107.0,0.014012,0.057718,0.141357,0.125363,0.153671,0.129621,0.118631,0.131013,0.086082,0.032448,0.010084
2,State/Province,North America,United States,West,Alaska,AK,2,2020-01-24,01/24/2020,0,2021-03-14 14:46:21.129247,0,0,0,0,0,0,712114.0,7.12114,327052602.0,3270.52602,365057840.0,3650.5784,7.796609e+09,77966.09105,0.002177,0.001951,0.000091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9978.0,10012.0,10186.0,10509.0,10395.0,10414.0,10303.0,10286.0,10436.0,10157.0,9976.0,10016.0,9887.0,9509.0,9678.0,9488.0,9410.0,9343.0,8518.0,7525.0,8088.0,8617.0,9132.0,9252.0,9900.0,10318.0,10693.0,11456.0,11576.0,11552.0,10946.0,10809.0,10460.0,10822.0,10799.0,10303.0,10452.0,9962.0,9667.0,9685.0,8865.0,8589.0,8533.0,7954.0,8295.0,7827.0,7962.0,8230.0,8696.0,9086.0,8428.0,8197.0,8330.0,8595.0,9128.0,9426.0,9493.0,9636.0,9731.0,9894.0,9373.0,9168.0,9161.0,8614.0,8800.0,8210.0,7734.0,7220.0,6655.0,6442.0,5978.0,5621.0,5482.0,4013.0,3945.0,3665.0,3459.0,2950.0,2677.0,2327.0,1971.0,1784.0,1586.0,1411.0,1277.0,7181.0,41102.0,100662.0,89273.0,109431.0,92305.0,84479.0,93296